# ComCam shutter open time

Craig Lage - 13-Nov-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy.visualization import time_support
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from astroplan import Observer
from pytz import timezone
import os
os.environ["no_proxy"] += ",.consdb"
from lsst.summit.utils import ConsDbClient
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay

In [ ]:
client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')
print(client.schema())  # list the instruments

In [ ]:
tz = timezone("Chile/Continental")
rubin = Observer(location=SIMONYI_LOCATION, name="Rubin", timezone=tz)

startDay = 20241024
endDay = 20241119

days = []
percents = []
dayObs = startDay
while dayObs <= endDay:
    query = 'SELECT exp_time, exposure_id, obs_start, obs_end '
    query += f'from cdb_lsstcomcam.exposure where day_obs = {dayObs}'
    data = client.query(query)
    dayObsString = dayObsIntToString(dayObs)
    eve_time = Time(f"{dayObsString}T20:00:00", scale='utc')
    eve = rubin.twilight_evening_nautical(eve_time)
    nextDayObs = calcNextDay(dayObs)  # find the next dayObs 
    nextDayObsString = dayObsIntToString(nextDayObs)
    morn_time = Time(f"{nextDayObsString}T08:00:00", scale='utc')
    morn = rubin.twilight_morning_nautical(morn_time)
    data = data[data['obs_start'] > eve]
    data = data[data['obs_end'] < morn]
    shutter_open = np.sum(data['exp_time'])
    total_time = (Time(data['obs_end'][-1], scale ='tai').jd - Time(data['obs_start'][0], scale='tai').jd) * 86400
    open_percent = shutter_open / total_time * 100.0
    #print(dayObs, open_percent)
    days.append(Time(f"{nextDayObsString}T00:00:00", scale='utc').isot)
    percents.append(open_percent)
    dayObs = nextDayObs
    
time_support()
plt.title("ComCam campaign shutter open percentage", fontsize=18)
plt.plot(Time(days, format='isot'), percents, marker='x')   
plt.ylabel("Shutter open percentage")
plt.savefig(f"/home/c/cslage/u/ComCam/data/Shutter_open_{endDay}.png")